In [1]:
seed=198

In [2]:
# importing required libraries
import torch.nn as nn
import torch
import torch.nn.functional as F
import math,copy,re
import warnings
import pandas as pd
import numpy as np
import random
import pickle
import torch.nn.init as init
from torchviz import make_dot
# import seaborn as sns
# import torchtext
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import display
import matplotlib.pyplot as plt
import os 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
warnings.simplefilter("ignore")
print(torch.__version__)

2.0.1+cpu


In [3]:
# Check if CUDA is available
if torch.cuda.is_available():
    # CUDA is available, so PyTorch can use a GPU
    device = torch.device("cuda")
    print("PyTorch is using GPU.")
else:
    # CUDA is not available, PyTorch will use CPU
    device = torch.device("cpu")
    print("PyTorch is using CPU.")

# Create a tensor and move it to the device
tensor = torch.randn(3, 4).to(device)

# Check the device of the tensor
print("Tensor device:", tensor.device)

PyTorch is using CPU.
Tensor device: cpu


In [4]:
def reset_random_seeds(seed):
    os.environ['PYTHONHASHSEED']=str(seed)
#     tf.random.set_seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [5]:
pickle_file_path = 'C:/Users/Asus/Downloads/Ten Minutes.pkl'

# Load data from the pickle file
with open(pickle_file_path, "rb") as f:
    loaded_data_dict = pickle.load(f)

# Retrieve data and labels
X_train = loaded_data_dict["X_train"]
y_train = loaded_data_dict["y_train"]
X_test = loaded_data_dict["X_test"]
y_test = loaded_data_dict["y_test"]

In [6]:
Valid, Test = X_test[:100000], X_test[100000:]
Valid_label, Test_label = y_test[:100000], y_test[100000:]

In [7]:
num_epochs = 500
# Define batch size
batch_size = 256
# Create DataLoader for training set
train_dataset = torch.utils.data.TensorDataset(X_train.float(), y_train.float())
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
valid_dataset = torch.utils.data.TensorDataset(Valid.float(), Valid_label.float())
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_dataset = torch.utils.data.TensorDataset(Test.float(), Test_label.float())
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [10]:
import torch
import torch.nn as nn
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 1024),
            nn.ReLU(), 
            nn.Linear(1024, 3), 
            nn.Softmax(dim=1)  
        )
        
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, _ = self.lstm(x, (h0.detach(), c0.detach()))
        
        # Index hidden state of last time step
        print(out.shape)
        out = out[:, -1, :]
        out = self.classifier(out)
        return out

# Hyperparameters
input_size = 13
hidden_size = 500
num_layers = 2
num_classes = 3
reset_random_seeds(seed)
# Initialize the model
model = LSTMModel(input_size, hidden_size, num_layers, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.000001)


# Define early stopping parameters
patience = 10
best_loss = 0.0
epochs_without_improvement = 0
train = 1
device = torch.device("cpu")
if train:
    # Step 4: Training loop
    for epoch in range(num_epochs):
        running_loss = 0.0
        latents = []
        LS = []

        # Iterate over mini-batches
        for i, (inputs, labels) in enumerate(train_loader):
            
            
            
            # Clear gradients
            optimizer.zero_grad()
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            
            # Calculate loss
            loss = criterion(outputs, labels)
            loss = loss.float()

            # Backpropagation
            loss.backward()
            optimizer.step()

            # Update running loss
            running_loss += loss.item()
            
            latents.append(outputs)
            LS.append(labels)
            

        # Calculate average loss per epoch
        epoch_loss = running_loss / len(train_loader)
        latents = torch.cat(latents, dim=0)
        LS = torch.cat(LS, dim=0)
        _, predicted = torch.max(latents.data, 1)
        _, lbl = torch.max(LS.data, 1)
        total = lbl.size(0)
        correct = (predicted == lbl).sum().item()

        accuracy = 100 * correct / total
        print(f'Train Accuracy: {accuracy:.2f}%')
        conf_matrix = confusion_matrix(lbl.cpu().detach().numpy(),predicted.cpu().detach().numpy())
        print('Confusion Matrix:')
        print(conf_matrix)
        
        if epoch%1==0:
            M,N = [],[]
            for j, (inp, lab) in enumerate(valid_loader):
                inp = inp.to(device)
                lab = lab.to(device)
                
                with torch.no_grad():
                    outp = model(inp)
                M.append(outp)
                N.append(lab)
                    
            M = torch.cat(M, dim=0)
            N = torch.cat(N, dim=0)
            _, pre = torch.max(M.data, 1)
            _, lb = torch.max(N.data, 1)
            total = lb.size(0)
            correct = (pre == lb).sum().item()
            accuracy = 100 * correct / total
            print('-----------------------------------------')
            print(f'Valid Accuracy: {accuracy:.2f}%')
            lb = lb.cpu().detach().numpy()
            pre = pre.cpu().detach().numpy()
            conf_matrix = confusion_matrix(lb,pre)
            precision = precision_score(lb, pre,average="macro")
            recall = recall_score(lb, pre,average="macro")
            f1 = f1_score(lb, pre,average="macro")
            print('Confusion Matrix:')
            print(conf_matrix)
            print()
            print("Precision:", precision)
            print("Recall:", recall)
            print("F1 Score:", f1)
            print('-----------------------------------------')
            # Log epoch progress (optional)
            print(f"Epoch: {epoch+1}, Loss: {round(epoch_loss,6)}")
            print()
            print()
        
        # Check for early stopping and save the best weights
        if f1 > best_loss:
            best_loss = f1
            epochs_without_improvement = 0
            # Save the model weights
            
            
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print("Early stopping triggered. Training stopped.")
                break

torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144, 500])
torch.Size([256, 144

KeyboardInterrupt: 